In [ ]:
import numpy as np
import pandas as pd

# تعریف تعداد نمونه‌ها
num_samples = 1000

# پارامترهای ثابت
np.random.seed(42)  # ثابت کردن Seed برای تولید قابل تکرار
MW = 16.04  # وزن مولکولی گاز (kg/kmol)
R = 8.314 / MW  # ثابت گاز جهانی برای گاز متان (kJ/kg·K)
gamma = 1.31  # نسبت گرمای مخصوص
Cp = gamma * R / (gamma - 1)  # گرمای مخصوص ثابت فشار (kJ/kg·K)

# تعریف بردار زمانی
time_interval = 1  # فاصله زمانی (ثانیه)
time = np.arange(0, num_samples * time_interval, time_interval)  # بردار زمانی

# تولید داده‌های سری زمانی
pressure_in = 3.5 + 0.2 * np.sin(0.05 * time) + np.random.normal(0, 0.1, num_samples)  # فشار ورودی (bara)
temperature_in = 293 + 5 * np.cos(0.03 * time) + np.random.normal(0, 2, num_samples)  # دمای ورودی (K)
flow_rate = 12 + 0.5 * np.sin(0.04 * time) + np.random.normal(0, 0.2, num_samples)  # جریان جرمی (kg/s)
efficiency = 0.82 + 0.02 * np.sin(0.02 * time) + np.random.normal(0, 0.01, num_samples)  # کارآیی چندگانه
vibration = 1.0 + 0.3 * np.sin(0.06 * time) + np.random.normal(0, 0.1, num_samples)  # ارتعاشات (mm/s)
status = np.where(vibration < 1.5, "Normal", np.where(vibration < 3.0, "Imbalance", "Fault"))  # وضعیت کمپرسور

# محاسبات مبتنی بر روابط فیزیکی
pressure_ratio = 5.0 + 0.2 * np.sin(0.05 * time) + np.random.normal(0, 0.1, num_samples)  # نسبت فشار
pressure_out = pressure_in * pressure_ratio  # فشار خروجی (bara)

# دمای خروجی
temperature_out = temperature_in * (pressure_ratio ** ((gamma - 1) / (gamma * efficiency)))

# توان مصرفی
power_consumption = flow_rate * Cp * (temperature_out - temperature_in) / efficiency  # kW

# داده‌های مرتبط با ارتعاشات
frequency = 50 + 10 * np.sin(0.03 * time) + np.random.normal(0, 5, num_samples)  # فرکانس ارتعاشات (Hz)
amplitude = 0.5 + 0.2 * np.sin(0.04 * time) + np.random.normal(0, 0.1, num_samples)  # Amplitude ارتعاشات (mm)
phase_angle = 180 + 30 * np.sin(0.02 * time) + np.random.normal(0, 10, num_samples)  # زاویه فاز (Degrees)

# داده‌های مرتبط با معادله جرم-فنر
mass = 75 + 5 * np.sin(0.01 * time) + np.random.normal(0, 2, num_samples)  # جرم محور (kg)
stiffness = 5e5 + 1e4 * np.sin(0.01 * time) + np.random.normal(0, 1e4, num_samples)  # سختی فنر (N/m)
damping = 500 + 50 * np.sin(0.01 * time) + np.random.normal(0, 20, num_samples)  # ضریب میرایی (Ns/m)

# داده‌های مرتبط با معادله ناویر-استوکس
density = 0.7 + 0.05 * np.sin(0.02 * time) + np.random.normal(0, 0.02, num_samples)  # چگالی گاز (kg/m³)
velocity = 30 + 5 * np.sin(0.03 * time) + np.random.normal(0, 2, num_samples)  # سرعت جریان (m/s)
viscosity = 1e-5 + 1e-6 * np.sin(0.02 * time) + np.random.normal(0, 1e-7, num_samples)  # ضریب-viscosity (Pa·s)

# ایجاد DataFrame
data = {
    "Time": time,  # اضافه کردن بردار زمانی
    "Pressure_In": pressure_in,
    "Temperature_In": temperature_in - 273.15,  # تبدیل به °C
    "Flow_Rate": flow_rate,
    "Pressure_Out": pressure_out,
    "Temperature_Out": temperature_out - 273.15,  # تبدیل به °C
    "Efficiency": efficiency,
    "Power_Consumption": power_consumption,
    "Vibration": vibration,
    "Status": status,
    "Frequency": frequency,
    "Amplitude": amplitude,
    "Phase_Angle": phase_angle,
    "Mass": mass,
    "Stiffness": stiffness,
    "Damping": damping,
    "Density": density,
    "Velocity": velocity,
    "Viscosity": viscosity
}

df = pd.DataFrame(data)

# ذخیره داده‌ها در فایل CSV
df.to_csv("compressor_time_series_data.csv", index=False)

print("داده‌های سینتیک پیشرفته با سری زمانی با موفقیت تولید و ذخیره شد.")

داده‌های سینتیک پیشرفته با سری زمانی با موفقیت تولید و ذخیره شد.


In [6]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

# بارگذاری داده‌ها
df = pd.read_csv("compressor_time_series_data.csv")

# ۱. بررسی و حذف داده‌های گمشده
numeric_cols = df.select_dtypes(include=[np.number]).columns  # انتخاب ستون‌های عددی
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())  # پر کردن مقدارهای گمشده با میانگین

# پر کردن مقادیر گمشده در ستون‌های متنی با مقدار پرتکرار
text_cols = df.select_dtypes(include=["object"]).columns
for col in text_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# ۲. حذف داده‌های تکراری
df.drop_duplicates(inplace=True)

# ۳. بررسی و حذف داده‌های پرت (Outliers) بر اساس Z-Score
z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
df = df[(z_scores < 3).all(axis=1)]  # حذف مقادیر پرت

# ۴. استانداردسازی داده‌های متنی
df["Status"] = df["Status"].str.lower().str.strip()

# ۵. بررسی تناقضات منطقی در داده‌ها
df = df[df["Pressure_Out"] >= df["Pressure_In"]]  # فشار خروجی نباید کمتر از ورودی باشد
df = df[df["Temperature_Out"] >= df["Temperature_In"]]  # دمای خروجی نباید کمتر از ورودی باشد

# ۶. نرمال‌سازی داده‌های عددی
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# ذخیره داده‌های تمیز شده در فایل جدید
df.to_csv("cleaned_compressor_data.csv", index=False)

print("داده‌ها با موفقیت تمیز و ذخیره شدند!")


داده‌ها با موفقیت تمیز و ذخیره شدند!


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10844\863847342.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [3]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller, kpss

# 📌 خواندن دیتاست
df = pd.read_csv("compressor_time_series_data.csv")

# 📌 لیست متغیرهای مورد بررسی
variables = ["Pressure_In", "Temperature_In", "Flow_Rate", "Vibration"]

# 📌 تابع برای اجرای آزمون ADF
def adf_test(series):
    result = adfuller(series.dropna())  # حذف مقادیر NaN
    return result[1]  # مقدار p-value را برمی‌گرداند

# 📌 تابع برای اجرای آزمون KPSS
def kpss_test(series):
    result = kpss(series.dropna(), regression="c")
    return result[1]  # مقدار p-value را برمی‌گرداند

# 📌 بررسی ایستایی قبل از تفاضل‌گیری
print("🔹 بررسی ایستایی قبل از تفاضل‌گیری")
for var in variables:
    adf_pval = adf_test(df[var])
    kpss_pval = kpss_test(df[var])

    print(f"\n🔸 متغیر: {var}")
    print(f"ADF p-value: {adf_pval} → {'✔ ایستا' if adf_pval < 0.05 else '❌ نایستا'}")
    print(f"KPSS p-value: {kpss_pval} → {'✔ ایستا' if kpss_pval > 0.05 else '❌ نایستا'}")

# 📌 ایجاد متغیرهای تفاضلی (در صورت نیاز)
for var in variables:
    df[f"{var}_Diff"] = df[var].diff()  # تفاضل مرتبه اول

# 📌 بررسی ایستایی بعد از تفاضل‌گیری
print("\n🔹 بررسی ایستایی بعد از تفاضل‌گیری")
for var in variables:
    adf_pval = adf_test(df[f"{var}_Diff"])
    print(f"\n🔸 متغیر: {var}_Diff")
    print(f"ADF p-value: {adf_pval} → {'✔ ایستا' if adf_pval < 0.05 else '❌ نایستا'}")

# 📌 ذخیره داده‌های جدید
df.to_csv("compressor_time_series_data_processed.csv", index=False)

print("\n✅ بررسی ایستایی کامل شد و داده‌های پردازش‌شده ذخیره شدند!")


🔹 بررسی ایستایی قبل از تفاضل‌گیری

🔸 متغیر: Pressure_In
ADF p-value: 2.0962876212431116e-06 → ✔ ایستا
KPSS p-value: 0.1 → ✔ ایستا

🔸 متغیر: Temperature_In
ADF p-value: 0.1469540188086274 → ❌ نایستا
KPSS p-value: 0.1 → ✔ ایستا

🔸 متغیر: Flow_Rate
ADF p-value: 2.5069358586127073e-05 → ✔ ایستا
KPSS p-value: 0.1 → ✔ ایستا

🔸 متغیر: Vibration
ADF p-value: 2.5874765662807433e-18 → ✔ ایستا
KPSS p-value: 0.1 → ✔ ایستا

🔹 بررسی ایستایی بعد از تفاضل‌گیری

🔸 متغیر: Pressure_In_Diff
ADF p-value: 1.204261423685234e-05 → ✔ ایستا

🔸 متغیر: Temperature_In_Diff
ADF p-value: 2.9490882379678414e-29 → ✔ ایستا

🔸 متغیر: Flow_Rate_Diff
ADF p-value: 0.0001231829724437467 → ✔ ایستا

🔸 متغیر: Vibration_Diff
ADF p-value: 0.0010076228294006095 → ✔ ایستا

✅ بررسی ایستایی کامل شد و داده‌های پردازش‌شده ذخیره شدند!


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10576\4142561126.py:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(series.dropna(), regression="c")
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10576\4142561126.py:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(series.dropna(), regression="c")
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10576\4142561126.py:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(series.dropna(), regression="c")
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10576\4142561126.py:18: InterpolationWarning: The test statistic is outside of the range of p-values

In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

# 📌 خواندن دیتاست
df = pd.read_csv("compressor_time_series_data.csv")

# 📌 انتخاب متغیرهای عددی (حذف ستون‌های غیرعددی مثل "Status")
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df_numeric = df[numeric_columns]

# 📌 مقیاس‌بندی Min-Max (بین 0 و 1)
minmax_scaler = MinMaxScaler()
df_minmax = pd.DataFrame(minmax_scaler.fit_transform(df_numeric), columns=numeric_columns)

# 📌 استانداردسازی (Z-Score)
standard_scaler = StandardScaler()
df_standard = pd.DataFrame(standard_scaler.fit_transform(df_numeric), columns=numeric_columns)

# 📌 نرمال‌سازی به مقدار واحد
normalizer = Normalizer()
df_normalized = pd.DataFrame(normalizer.fit_transform(df_numeric), columns=numeric_columns)

# 📌 ذخیره دیتاست‌های پردازش‌شده
df_minmax.to_csv("compressor_minmax_scaled.csv", index=False)
df_standard.to_csv("compressor_standard_scaled.csv", index=False)
df_normalized.to_csv("compressor_normalized.csv", index=False)

print("✅ مقیاس‌بندی انجام شد و فایل‌های جدید ذخیره شدند! 🚀")


✅ مقیاس‌بندی انجام شد و فایل‌های جدید ذخیره شدند! 🚀


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 📌 خواندن دیتاست
df = pd.read_csv("compressor_time_series_data.csv")

# 📌 جدا کردن ویژگی‌ها (X) و خروجی (y) - در اینجا فرض می‌کنیم متغیر "Efficiency" خروجی است
X = df.drop(columns=["Efficiency", "Status"])  # حذف خروجی و متغیر دسته‌بندی
y = df["Efficiency"]

# 📌 تقسیم داده‌ها به (Train 70%) و (Test 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 📌 تقسیم داده‌های باقی‌مانده به (Validation 20%) و (Test 10%)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.33, random_state=42)  # 33% از 30% = 10%

# 📌 نمایش اندازه هر مجموعه
print(f"✅ تعداد نمونه‌های آموزشی: {X_train.shape[0]}")
print(f"✅ تعداد نمونه‌های اعتبارسنجی: {X_val.shape[0]}")
print(f"✅ تعداد نمونه‌های تست: {X_test.shape[0]}")

# 📌 ذخیره داده‌های تقسیم‌شده
X_train.to_csv("X_train.csv", index=False)
X_val.to_csv("X_val.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_val.to_csv("y_val.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

print("✅ داده‌ها تقسیم و ذخیره شدند! 🚀")


✅ تعداد نمونه‌های آموزشی: 700
✅ تعداد نمونه‌های اعتبارسنجی: 201
✅ تعداد نمونه‌های تست: 99
✅ داده‌ها تقسیم و ذخیره شدند! 🚀
